In [23]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

2025-04-19 14:56:52.089645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745074612.259891      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745074612.310007      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [24]:
# Load the preprocessed dataset
df = pd.read_csv("/kaggle/working/preprocessed_dataset.csv")

# Define features (X) and labels (y)
X = df.drop(columns=['attack'])  # Drop the target column
y = df['attack']  # Target column

# Split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for CNN input (samples, timesteps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [26]:
def build_cnn_lstm(input_shape):
    inputs = Input(shape=input_shape)

    # CNN Layer for feature extraction
    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
    x = Dropout(0.3)(x)

    # LSTM Layer for sequential data processing
    x = LSTM(64, return_sequences=True)(x)
    x = Dropout(0.3)(x)

    # Fully connected layers (use LSTM output directly)
    x = Flatten()(x)    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)  # Binary classification (Benign vs Attack)

    model = Model(inputs, outputs)
    return model

In [27]:
# Define input shape based on training data
input_shape = (X_train.shape[1], X_train.shape[2])

# Build and compile the model
model = build_cnn_lstm(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

I0000 00:00:1745074842.849100      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 19, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 19, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 19, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 19, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 19, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 19, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         155,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 230,273 (899.50 KB)

 Trainable params: 230,273 (899.50 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Add EarlyStopping to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=128,
    callbacks=[early_stop]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=128)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Save the model
model.save("/kaggle/working/cnn_lstm_model.h5")

Epoch 1/20


I0000 00:00:1745074884.922254     138 cuda_dnn.cc:529] Loaded cuDNN version 90300


138536/138536 ━━━━━━━━━━━━━━━━━━━━ 874s 6ms/step - accuracy: 0.5470 - loss: 0.6867 - val_accuracy: 0.5481 - val_loss: 0.6856
Epoch 2/20
138536/138536 ━━━━━━━━━━━━━━━━━━━━ 898s 6ms/step - accuracy: 0.5479 - loss: 0.6858 - val_accuracy: 0.5491 - val_loss: 0.6853
Epoch 3/20
138536/138536 ━━━━━━━━━━━━━━━━━━━━ 852s 6ms/step - accuracy: 0.5485 - loss: 0.6855 - val_accuracy: 0.5492 - val_loss: 0.6849
Epoch 4/20
138536/138536 ━━━━━━━━━━━━━━━━━━━━ 840s 6ms/step - accuracy: 0.5489 - loss: 0.6853 - val_accuracy: 0.5501 - val_loss: 0.6849
Epoch 5/20
138536/138536 ━━━━━━━━━━━━━━━━━━━━ 841s 6ms/step - accuracy: 0.5489 - loss: 0.6852 - val_accuracy: 0.5504 - val_loss: 0.6843
Epoch 6/20
138536/138536 ━━━━━━━━━━━━━━━━━━━━ 839s 6ms/step - accuracy: 0.5491 - loss: 0.6851 - val_accuracy: 0.5506 - val_loss: 0.6843
Epoch 7/20
138536/138536 ━━━━━━━━━━━━━━━━━━━━ 834s 6ms/step - accuracy: 0.5493 - loss: 0.6850 - val_accuracy: 0.5506 - val_loss: 0.6843
Epoch 8/20
 87783/138536 ━━━━━━━━━━━━━━━━━━━━ 4:38 5ms/step

In [ ]:
# CNN Layer:
# Input shape: (None, 20, 1) → 20 timesteps with 1 feature
# Conv1D Layer:
# - Filters: 64
# - Kernel size: 3
# - Activation: ReLU
# - Padding: same
# - Strides: 1
# - Input channels: 1 (from input shape)
# - Output channels: 64 (filters)
# - Output shape: (None, 20, 64) → 64 filters for each of the 20 timesteps
# Params = (kernel_size × input_channels + bias) × filters = (3×1 + 1) × 64 = 256
# Conv1D Layer:
# - Filters: 128
# - Kernel size: 3
# - Activation: ReLU
# - Padding: same
# - Strides: 1
# - Input channels: 64 (from previous layer)
# - Output channels: 128 (filters)
# - Output shape: (None, 20, 128) → 128 filters for each of the 20 timesteps
# Params = (kernel_size × input_channels + bias) × filters = (3×64 + 1) × 128 = 24,704
# Dropout Layer:
# - Dropout rate: 0.3
# - Input shape: (None, 20, 128) → 128 features for each of the 20 timesteps
# - Output shape: (None, 20, 128) → same as input shape
# LSTM Layer:
# - Units: 64
# - Return sequences: True
# - Input shape: (None, 20, 128) → 128 features for each of the 20 timesteps
# - Output shape: (None, 20, 64) → 64 hidden units for each of the 20 timesteps
# Params = 4 × [(input_dim + units) × units + units] = 4 × [(128 + 64) × 64 + 64] = 49,408


